# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [11]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Modulos da solucao
import modules.cronometro as cron
import modules.utils as utl
import modules.trata_pdf as tpdf


In [2]:
#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"

# 4. XXX Tipos de documentos para extracao
tipo_documento_path = "config/tipo_documentos"

# 5. XXX Path para tipo de documento patterns
tipo_documento_patterns_path = "config/tipo_documentos/patterns"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"



#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

### Funcoes para Analise do pipeline

In [48]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/159871/2023 -6.pdf"
nome_arquivo = os.path.basename(file_path)

In [49]:
pdf_document = fitz.open(file_path)

# Página do PDF  ATENCAO  (UNICA PAGINA)
page_number = 0  # Defina o número da página que deseja analisar
page = pdf_document[page_number]

# Extrair texto dentro do retângulo
text_P = page.get_text("text")

pdf_document.close()

texto_PDF = re.sub('\s+', ' ', text_P).strip()  
texto_PDF

"27/07/2023, 15:18 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20236 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:16:00 Código Verificação: 3650A24CE PRESTADOR DE SERVIÇOS CPF/CNPJ: 50.921.369/0001-05 Inscrição Municipal: 952538 Telefone: 2297268232.. Inscrição Estadual: Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ: 06.047.087/0033-16 | INSC:MUNICIPAL: 1664085 RG: Telefone: Inscrição Estadual: Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço: OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Março, 48h no Setor de Radiologia

In [52]:
valor = 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e'
valor.lower()

'nota fiscal de serviços eletrônica - nfs-e'

In [ ]:
NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e

In [3]:
sequence = 1
tipo_documento = "nfs_e"
tipo_documento_descricao = "nota fiscal de serviços eletrônica - nfs-e"
conteudo_padrao = """27/07/2023, 15:18 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20236 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:16:00 Código Verificação: 3650A24CE PRESTADOR DE SERVIÇOS CPF/CNPJ: 50.921.369/0001-05 Inscrição Municipal: 952538 Telefone: 2297268232.. Inscrição Estadual: Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ: 06.047.087/0033-16 | INSC:MUNICIPAL: 1664085 RG: Telefone: Inscrição Estadual: Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço: OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Março, 48h no Setor de Radiologia - Médica: Lara Veiga Soria Catuladeira. VALOR TOTAL DA NOTA: R$ 5.706,88 CNAE - 8630502 - ATIVIDADE MÉDICA AMBULATORIAL COM RECURSOS PARA REALIZAÇÃO DE EXAMES COMPLEMENTARES Item da Lista de Serviços - 4.03 - HOSPITAIS, CLÍNICAS, LABORATÓRIOS, SANATÓRIOS, MANICÔMIOS, CASAS DE SAÚDE, PRONTOS-SOCORROS, AMBULATÓRIOS E CONGÊNERES. VALOR SERVIÇOS: R$ 5.706,88 VALOR DEDUÇÃO: R$ 0,00 DESC. INCOND: R$ 0,00 BASE DE CÁLCULO: R$ 5.706,88 ALÍQUOTA: 2,01% VALOR ISS: R$ 114,71 VALOR ISS RETIDO: R$ 0,00 DESC. COND: R$ 0,00 ____________________________________________________________________ VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR INSS: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS RETENÇÕES: R$ 0,00 VALOR LÍQUIDO: R$ 5.706,88 DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Sim ( 2,01% ) ISSQN RETIDO Não LOCAL. PRESTAÇÃO SERVIÇO Mesquita - RJ LOCAL INCIDÊNCIA Mesquita - RJ Observação: LEI DA TRANSPARÊNCIA FISCAL NR. 12.741, DE 8 DE DEZEMBRO DE 2012. - PRESTADOR OPTANTE DO SIMPLES NACIONAL (ALÍQUOTA: 2,01 %) Valor Aproximado dos Tributos Federais R$ 767,58 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 127,26 (Alíq IBPT 2,23 IBPT)"""

In [17]:
# XXX Criar um dicionário para matchers patterns - nfs-e
matcher_pattern_dict = {
        "numero_nota_pattern": [
            {"LOWER": "número"},
            {"LOWER": "da"},
            {"LOWER": "nota"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True}
        ],
        "competencia_pattern": [
            {"LOWER": "competência"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "?"},
            {"ORTH": {"REGEX": "^[A-Z][a-z]+/[0-9]{4}$"}}   
        ],
        "data_hora_emissao_pattern": [
            {"LOWER": "data"},
            {"LOWER": "e"},
            {"LOWER": "hora"},
            {"LOWER": "da"},
            {"LOWER": "emissão"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"SHAPE": "dd/dd/dddd"},
            {"IS_SPACE": True, "OP": "*"},
            {"SHAPE": "dd:dd:dd"}
        ],
        "codigo_verificacao_pattern": [
            {"LOWER": "código"},
            {"LOWER": "verificação"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ASCII": True, "LENGTH": 9}
        ],
        "valor_total_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "total"},
            {"LOWER": "da", "OP": "?"},
            {"LOWER": "nota", "OP": "?"},
            {"TEXT": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_servicos_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "serviços"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_deducao_pattern": [
            {"LOWER": "dedução"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_incondR_pattern": [
            {"LOWER": "base"},
            {"LOWER": "de"},
            {"IS_SPACE": True},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"ORTH": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}    
        ],
        "valor_incond_patternP": [
            {"LOWER": "desc"},
            {"IS_PUNCT": True, "OP": "?"},
            {"LOWER": "incond"},
            {"ORTH": ":"},
            {"IS_SPACE": True},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"ORTH": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}    
        ],
        "valor_calculoR_pattern": [
            {"LOWER": "cálculo"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_calculoP_pattern": [
            {"LOWER": "base"},
            {"LOWER": "de"},
            {"LOWER": "cálculo"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_aliquota_pattern": [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d,dd", "OP": "?"},
            {"ORTH": "%"}
        ],
        "valor_aliquota2_pattern": [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d", "OP": "?"},
            {"ORTH": "%"}
        ],
        "valor_iss_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_issretido_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"LOWER": "retido"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_desccond_pattern": [
            {"LOWER": "desc"},
            {"ORTH": "."},
            {"LOWER": "cond"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_pis_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "pis"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_cofins_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "cofins"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_ir_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "ir"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_inss_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "inss"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_csll_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "csll"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_outrasreten_pattern": [
            {"LOWER": "outras"},
            {"LOWER": "retenções"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_liquido_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "líquido"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "exigibilidade_iss_pattern": [
            {"LOWER": "exigibilidade"},
            {"LOWER": "iss"},
            {"LOWER": {"IN": ["exigivel", "não exigivel"]}}
        ],
        "padrao_regime_tributacao": [
            {"LOWER": "regime"},
            {"LOWER": "tributação"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ALPHA": True, "OP": "+"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ALPHA": True, "OP": "*"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "simples", "OP": "?"},
            {"IS_ALPHA": True, "OP": "*"}
        ],
        "simples_nacional_nao_pattern": [
            {"LOWER": "simples"},
            {"LOWER": "nacional"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "não"}
        ],
        "simples_nacional_pattern": [
            {"LOWER": "simples"},
            {"LOWER": "nacional"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "sim", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": "(", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": ",", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": "%", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": ")", "OP": "?"}
        ],
        "issqn_retido_pattern": [
            {"LOWER": "issqn"},
            {"LOWER": "retido"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": {"IN": ["sim", "não"]}}
        ],
        "local_prestacao_servico_pattern": [
            {"LOWER": "local"},
            {"ORTH": "."},
            {"LOWER": "prestação"},
            {"LOWER": "serviço"},
            {"IS_SPACE": True, "OP": "+"},  # para lidar com múltiplos espaços
            {"IS_ALPHA": True, "OP": "+"},  # para a cidade
            {"ORTH": "-", "OP": "?"},
            {"IS_UPPER": True, "LENGTH": 2, "OP": "?"}  # para a sigla do estado
        ],
        "local_incidencia_pattern": [
            {"LOWER": "local"},
            {"IS_PUNCT": True, "OP": "?"},
            {"LOWER": "incidência"},
            {"IS_ALPHA": True, "OP": "+"},  # Nome da cidade
            {"ORTH": "-", "OP": "?"},  # Hífen opcional
            {"SHAPE": "XX", "OP": "?"}  # Sigla do estado
        ],
        "valor_aliquota_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "+"},
            {"ORTH": "", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"},
            {"ORTH": "%"}
        ]
}    

In [19]:
nome_matches_pattern_json = tipo_documento + "_matcher_pattern.json"
path_matches_pattern_json = os.path.join(tipo_documento_patterns_path, nome_matches_pattern_json)
with open(path_matches_pattern_json, "w") as f:
    json.dump(matcher_pattern_dict, f)


'config/tipo_documentos/patterns/nfs_e_matcher_pattern.json'

In [ ]:




colors = {
            "secretaria": "linear-gradient(90deg, #2ADB5E, #1FA346)", # Verde Degrade
            "tipo_documento": "linear-gradient(90deg, #09D6FF, #08A0D1)", #Azul medio degrade
            "nome_prefeitura": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", # Roxo claro para lilaz - degrade bem bacana
            "nome_section": "linear-gradient(90deg, #FFA9FB, #BF7FBC)", #  lilaz - Degrade
            "nome_section": "#FFEA7F", # Laranja claro
            "SAFRA": "#CCA10C", # Terracota
            "SAFRA": "#AB9BFC", # Roxo claro 
            "CNPJ": "#7AECEC", # Azul bem claro
            "NOME": "#EE8AF8" # Rosa medio
        }          

patternsPrefeitura = [
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mesquita"}], "id": "PM_MESQUITA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mage"}], "id": "PM_MAGE"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "de"}, {"LOWER": "aldeia"}], "id": "PM_SPA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "da"}, {"LOWER": "aldeia"}], "id": "PM_SPA"}

                        ]


patternsSection = [     
                        {"label": "nome_section", "pattern": [{"LOWER": "número"}, {"LOWER": "da"}, {"LOWER": "nota"}, {"ORTH": ":"}], "id": "1. CABECALHO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "discriminação"}, {"LOWER": "dos"}, {"LOWER": "serviços"}], "id": "4. DESCRIMINACAO DOS SERVIÇOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "total"}, {"LOWER": "da"}, {"LOWER": "nota"}], "id": "5. VALOR TOTAL"},
                        {"label": "nome_section", "pattern": [{"LOWER": "cnae"}], "id": "6. CNAE e Item da Lista de Serviços"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "serviços"}], "id": "7. VALORES E IMPOSTOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "dados"}, {"LOWER": "complementares"}], "id": "8. DADOS COMPLEMENTARES"},
                        {"label": "nome_section", "pattern": [{"LOWER": "outras"}, {"LOWER": "informações"}, {"IS_PUNCT": True}, {"LOWER": "criticas"}], "id": "9. OUTRAS INFORMAÇOES / CRITICAS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "observação"}], "id": "10. OBSERVACOES"}

                        ]


patternsSecretarias = [{"label": "secretaria", "pattern": [{"LOWER": "secretaria"}, {"LOWER": "municipal"}, {"LOWER": "da"}, {"LOWER": "fazenda"},], "id": "SECRETARIA"}] 


patternsTipoDocumento = [
                        {"label": "tipo_documento", "pattern": [{"LOWER": "nota"}, {"LOWER": "fiscal"}, {"LOWER": "de"}, {"LOWER": "serviços"}, {"LOWER": "eletrônica"}, {"LOWER": "-"}, {"LOWER": "nfs-e"}], "id": "NFS-e"}
                        ]


patternsIdentificaEntidade = [
                            {"label": "CNPJ", "pattern": [{"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$"}}], "id": "cpf_cnpj_com_mascara"}
                            ]



patternsCnpj = [
    {
        "label": "CNPJ",
        "pattern": [
            {"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/$"}},
            {"ORTH": {"REGEX": "^\d{4}-\d{2}$"}}
        ]
    }
]




patternsTemp = [
                    {"label":"TOTAL", "pattern": [{"LOWER": "quantidade", "OP":"?"}, {"LOWER": "total", "OP":"*"}], "id": "qtde-total"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entregue","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entreguei","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "foram", "OP":"?"},{"LOWER": "entregues","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"SALDO", "pattern": [{"LOWER": "saldo","OP":"*"}], "id": "qtde-saldo"}]


patternsOthers = [{"label": "PERSON", "pattern": "Daniel", "id": "pessoa-daniel"}] 
 
patternsCult = [
    {
        "label":"CULTURA",
        "pattern": [
            {"LOWER": "soja", "OP":"?"},
            {"LOWER": "milho", "OP":"?"},
            {"LOWER": "sorgo", "OP":"?"},
            {"LOWER": "trigo", "OP":"?"},
            {"LOWER": "milheto", "OP":"?"},  
            
        ],    
    "id": "cultura"}]

patternsQuant = [{"label":"TOTAL", "pattern": [{"LOWER": "quantidade", "OP":"?"}, {"LOWER": "total", "OP":"*"}], "id": "qtde-total"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entregue","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entreguei","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "foram", "OP":"?"},{"LOWER": "entregues","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"SALDO", "pattern": [{"LOWER": "saldo","OP":"*"}], "id": "qtde-saldo"}]


patternsSafra = [{"label":"SAFRA", "pattern": [{"LOWER": "safra", "OP":"?"},{"LOWER": "safras", "OP":"?"}], "id": "safra"}]


patternsNroSafra = [{"label":"NR_SAF", "pattern": [{"SHAPE": "dd/dd", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "próxima", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "passada", "OP":"*"}], "id": "nro_safra"}]   

patternsCliente = [{"label": "CLIENTE", "pattern": [{"LOWER": "berdinazzi"}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "lopito"}], "id": "cli-lopito"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "bungue"}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["bunge", "bongue", "bumgue"]}}}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["berdinazi"]}}}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["matarazzo"]}}}], "id": "cli-matarazzo"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["mezenga"]}}}], "id": "cli-mezenga"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "rei"}, {"LOWER": "do"}, {"LOWER": "gado"}], "id": "cli-reidogado"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["rei-do-gado"]}}}], "id": "cli-reidogado"},   
                   ]


patternsContrato = [{"label": "CONTRATO", "pattern": [{"LOWER": "contrato", "OP":"?"}], "id": "contrato"},
                    {"label": "CONTRATO", "pattern": [{"LOWER": "contratos", "OP":"?"}], "id": "contrato"}]

patternsNroContrato = [{"label": "NR_CONT", "pattern": [{"SHAPE": "dddX", "OP":"*"}], "id": "nro_contrato"},
                       {"label": "NR_CONT", "pattern": [{"POS": "NUM", "SHAPE": "ddd", "OP": "*"},
                                                            {"POS": "PROPN", "SHAPE": "X", "OP": "*"}], "id": "nro_contrato"}]

patternsIntent = [{"label": "INTENT", "pattern": [{"IS_TITLE": True, "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"POS": "ADJ", "OP":"*"}, {"POS": "VERB", "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"TEXT": {"FUZZY": {"IN": ["preciso", "gostaria", "informar"]}}}], "id": "user-intent"}]


patterns = patternsPrefeitura + patternsSection + patternsSecretarias + patternsTipoDocumento + patternsIdentificaEntidade + patternsCnpj

In [ ]:
# Carregar padrões do disco
with open("patterns.json", "r") as f:
    loaded_patterns = json.load(f)

# Adicionar ao Matcher
for label, pattern in loaded_patterns.items():
    matcher.add(label, [pattern])


In [42]:
texto_PDF = "Número da Nota: PREFEITURA MUNICIPAL DE MAGE já Competência: SECRETARIA MUNICIPAL DA FAZENDA Julho/2023 NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Data e Hora da Emissão: 26/07/2023 11:42:00] Código Verificação: 8A10EE0EF PRESTADOR DE SERVIÇOS CPF/CNPJ: 06.014.264/0001-80 Inscrição Municipal: 20525 Telefone: Inscrição Estadual: 2125160368. Nome/Razão n” a Social: TECHMUNIZ SOLUÇÕES TILTDA. Nome de Fantasia: Endereço: PÇA DR. NILO PECANHA ,137 SL 101 PARTE ,CENTRO - Magé-RJ E-mail: REGISTROGQLAFSCONTABILIDADE.COM.BR TOMADOR DE SERVIÇOS CPF/CNPJ: 01.369.056/0002-61 | | INSC:MUNICIPAL: RG: Telefone: Inscrição Estadual: Nome/Razão Social: SYS MANAGER INFORMATICA Endereço: LTDA. ALMIRANTE JÚLIO DE SÁ BIERRENBACH Nº 200 BLOCO 1B SALAS 101B 102B 121B 122B 123B 124B 125B BAIRRO: JACAREPAGUÁ CIDADE: RIO DE JANEIRO E-mail: NÃO INFORMADO - RJ CEP: 22775028 DISCRIMINAÇÃO DOS SERVIÇOS Prestação de serviço de instalação de Software - Documento emitido por ME ou EPP optante pelo Simples Nacional Prestação de serviços realizados em julho/2023 Dados bancários Banco: 0260 - Nu Pagamentos S.A. Agência: 0001 C/C: 31108642-3 Chave PIX: 06.014.264/0001-80 VALOR TOTAL DA NOTA: R$ 14.978,88 CNAE - 6209100 - SUPORTE TÉCNICO, MANUTENÇÃO E OUTROS SERVIÇOS EM TECNOLOGIA DA INFORMAÇÃO Item da Lista de Serviços - 1.07 - SUPORTE TÉCNICO EM INFORMÁTICA, INCLUSIVE INSTALAÇÃO, CONFIGURAÇÃO E MANUTENÇÃO DE PROGRAMAS DE COMPUTAÇÃO E BANCOS DE DADOS. [m]' E [=] a, Th nã VALOR SERVIÇOS: R$ 14.978,88 VALOR | DEDUÇÃO: R$ 0,00 VALOR PIS: R$ 0,00 VALOR COFINS: VALOR IR: R$ 0,00 R$ 0,00 DESC. INCOND: BASE DE R$ 0,00 CÁLCULO: R$ 14.978,88 ALÍQUOTA: 2,01% — VALORISS: R$ 301,08 VALOR ISS RETIDO: R$ 0,00 DESC. COND: R$ 0,00 do dh, : VALOR INSS: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS RETENÇÕES: R$ 0,00 VALOR LÍQUIDO: R$ 14.978,88 DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel Observação: REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Sim (2,01% ) ISSQN RETIDO Não LOCAL. PRESTAÇÃO SERVIÇO Magé - RJ LOCAL INCIDÊNCIA Magé - RJ - Prestador Optante do Simples Nacional (Alíquota: 2,01 %) Valor Aproximado dos Tributos Federais R$ 2014,66 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 441,88 (Alíq IBPT 2,95 IBPT)"

In [46]:
# Documento médio para NFSe e um exemplo de documento
documento_medio_nfse = ["27/07/2023, 15:18 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20236 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:16:00 Código Verificação: 3650A24CE PRESTADOR DE SERVIÇOS CPF/CNPJ:  50.921.369/0001-05 Inscrição Municipal:  952538 Telefone:  2297268232.. Inscrição Estadual:   Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ:  06.047.087/0033-16    |     INSC:MUNICIPAL: 1664085 RG:   Telefone: Inscrição Estadual:   Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço:  OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Março, 48h no Setor de Radiologia - Médica: Lara Veiga Soria Catuladeira. VALOR TOTAL DA NOTA: R$ 5.706,88 CNAE - 8630502 - ATIVIDADE MÉDICA AMBULATORIAL COM RECURSOS PARA REALIZAÇÃO DE EXAMES COMPLEMENTARES Item da Lista de Serviços - 4.03 - HOSPITAIS, CLÍNICAS, LABORATÓRIOS, SANATÓRIOS, MANICÔMIOS, CASAS DE SAÚDE, PRONTOS-SOCORROS, AMBULATÓRIOS E CONGÊNERES.   VALOR SERVIÇOS: R$ 5.706,88 VALOR DEDUÇÃO: R$ 0,00 DESC. INCOND: R$ 0,00 BASE DE CÁLCULO: R$ 5.706,88 ALÍQUOTA: 2,01% VALOR ISS: R$ 114,71 VALOR ISS RETIDO: R$ 0,00 DESC. COND: R$ 0,00 ____________________________________________________________________ VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR INSS: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS RETENÇÕES: R$ 0,00 VALOR LÍQUIDO: R$ 5.706,88 DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Sim ( 2,01% ) ISSQN RETIDO Não LOCAL. PRESTAÇÃO SERVIÇO Mesquita - RJ LOCAL INCIDÊNCIA Mesquita - RJ Observação: LEI DA TRANSPARÊNCIA FISCAL NR. 12.741, DE 8 DE DEZEMBRO DE 2012. - PRESTADOR OPTANTE DO SIMPLES NACIONAL (ALÍQUOTA: 2,01 %) Valor Aproximado dos Tributos Federais R$ 767,58 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 127,26 (Alíq IBPT 2,23 IBPT) "]
documento_exemplo = [texto_PDF]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documento_medio_nfse + documento_exemplo)

# Calculando Similaridade de Cosseno
similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

print(f"Score de Similaridade: {similarity_scores[0][0]}")


Score de Similaridade: 1.0000000000000002


In [10]:
# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [4]:
# 1. XXX apagar arquivos zone
utl.apagar_zone(documentos_extracao_path)

In [11]:
# SOMENTE PARA TESTE DE FUNCAO
#fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)
batch_name

'Batch_23'

In [12]:
# 0.A Extracao de texto de todo o documento - PDF PESQUISAVEL	
def analise_texto_PDF(i, document_unique_id, nome_arquivo, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF = text_P.replace('\n', ' ') 
    if debug:
        print(f'\nFUNC analise_texto_PDF: doc.:{nome_arquivo}   texto_PDF_P: \n\n{texto_PDF}\n\n')

    return texto_PDF

In [24]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            pdf_pesquisavel_map = None
            document_unique_id = utl.generate_unique_id()
            nome_arquivo = file
            
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
                compressed_file_path = file_path
                compressed_file_name = os.path.basename(compressed_file_path)
                compressed_file_hash = utl.generate_file_hash(compressed_file_path)
                compressed_file_unique_id = utl.generate_unique_id()
                parent_document_unique_id = compressed_file_unique_id
                level = 2
            new_path_name = os.path.join(output_dir, file)
            
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = tpdf.analisa_nro_pages(file_path)
                
                one_page = doc_one_page
                num_page = nro_pgs
                
                level = 3
                
                texto_PDF = analise_texto_PDF(i, document_unique_id, nome_arquivo, file_path, debug)
                
                if texto_PDF:
                    pdf_pesquisavel_map = True
                else:
                    pdf_pesquisavel_map = False
            
            else:
                one_page = False
                num_page = 0    
            # if doc_one_page:
            #     one_page = doc_one_page
            # else:
            #     one_page = False    
            #             rotulo = 'pdf_mul_paginas'
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": num_page,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "pdf_pesquisavel": pdf_pesquisavel_map,
                "document_unique_id": document_unique_id,
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": utl.generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [25]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []
debug = False

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status, debug)

In [26]:
# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,pages,palavra_chave,document_tag,action_item,level,pdf_pesquisavel,parent_document_unique_id,file_hash,file_path,informations
document_unique_id,,,,,,,,,,,,,,,,,,,,
e1d5d841-495d-4c44-adc6-ba54826ed6fa,1,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,MESQUITA_PDF_31282023_2258.zip,root_dir,False,0,zip,doc_zip,NO_PROCESS,2,None,3026c52e-5f56-4b00-aa9c-e56ae0bab842,8d7038d712373364fa4c7680a887a0ceed01c8692d6958...,pipeline_extracao_documentos/2_documentos_para...,
9e63ff13-7118-4dd6-96fa-b569911cbd4c,2,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,1.pdf,teste,True,1,default,prov_nota_fiscal,PROCESS,3,False,3026c52e-5f56-4b00-aa9c-e56ae0bab842,66a7db9ee1500d5f9fa5da26563cfd7b68f1f5ba3daba2...,pipeline_extracao_documentos/2_documentos_para...,
5f506133-5477-4f7a-b340-6955f1accb3f,3,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,Livro de Registro do ISSQN.pdf,115964,False,4,livro,prov_livro_registro,NO_PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,b960962503987f6e05f5646d71a789facfe4e80ccb8890...,pipeline_extracao_documentos/2_documentos_para...,
4d676b5b-d8e2-43fb-b07a-d8cb24849b87,4,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -5.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,23a28a363c2d2c8b700ac4775164f7c0f0e2d6cef6166d...,pipeline_extracao_documentos/2_documentos_para...,
30a26799-1e90-43e9-8cda-7678267e6f9d,5,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -7.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,54045f4c09341d9f8d69438e7afe71eff46bb4e731392b...,pipeline_extracao_documentos/2_documentos_para...,
530c276f-3e26-43b7-8af4-8dc99d06934f,6,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -4.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,ddd09bb806dc79e98a74c5cf1adc6a5bd23ea1b4f1bfa7...,pipeline_extracao_documentos/2_documentos_para...,
de2dee5a-6fb8-46bd-998d-7dcaf9868c5f,7,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -6.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,8910a092d3aa53b6a1eb805c783245493760a1c46dadfa...,pipeline_extracao_documentos/2_documentos_para...,
ce63f138-edb6-4f2d-9499-4f1d51763192,8,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -3.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,25387d066a46925acba7ddfe3cd97e2e70f92838ef4312...,pipeline_extracao_documentos/2_documentos_para...,
ada30046-247f-455a-b054-cf330917027f,9,26/09/2023 07:59:53,Batch_23,analise,scan_analise,root_analise,Analise,2023 -8.pdf,159871,True,1,default,prov_nota_fiscal,PROCESS,3,True,3026c52e-5f56-4b00-aa9c-e56ae0bab842,8be881c377bf3064ef61f014fd14602cc736a73b69de8c...,pipeline_extracao_documentos/2_documentos_para...,


### exportando o map_analise_path

In [48]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)

## Pesquisas e outros

### Atualizando o valor da coluna status_documento

In [44]:
def update_status(row):
    if row['seq'] in sequencia:
        row['status_documento'] = 'PREPROCESS_EXTRACT'
    return row

In [45]:
sequencia = [2, 8, 6]

In [46]:
df_root_pipe = df_root_pipe.apply(update_status, axis=1)
df_root_pipe

In [ ]:
# Podemos chegar de forma mais rapida
subset_df.loc[subset_df['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [33]:
# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)


# XXX Criando uma nova coluna no DF
df_conferencia.insert(loc=50, column='original_file_name', value=df_conferencia['file_path'].apply(lambda x: os.path.basename(x)))

df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]



# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')


# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)


# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]




registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']



df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)


# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)

NameError: name 'df_conferencia' is not defined